<a href="https://colab.research.google.com/github/Magguire/Magguire/blob/main/Pandaswk4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Electric Car usage Analysis using Python 

## Importing our libraries

In [162]:
# importing pandas
import pandas as pd
# importing numpy
import numpy as np

## Loading our dataset

In [163]:
# In this week's independent project, you will be working as a data scientist working for 
# an electric car-sharing service company. You have been tasked to process stations data to understand
# electric car usage over time using the dataset from 'http://bit.ly/autolib_dataset'

url = 'http://bit.ly/autolib_dataset'
#file_id = split('/')[-2]
electric_cars = pd.read_csv(url, sep=',')

# Preview our dataset

In [164]:
# Display the first 5 rows of our datset
electric_cars.tail()

,Address,Cars,Bluecar counter,Utilib counter,Utilib 1.4 counter,Charge Slots,Charging Status,City,Displayed comment,ID,Kind,Geo point,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute
4995,8 avenue MÃ©nelotte,2,2,0,0,0,nonexistent,Colombes,NaN,colombes-menelotte-8,STATION,"48.9246525, 2.259313",92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,4,0,0,1,operational,Paris,NaN,paris-dantzig-37,STATION,"48.8335103, 2.2987201",75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,1,0,0,1,operational,Paris,NaN,paris-bac-142,STATION,"48.8508194, 2.3237968",75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,2,0,0,0,nonexistent,Fontenay-Sous-Bois,NaN,fontenaysousbois-valdefontenay-2,STATION,"48.8528247, 2.4869085",94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27
4999,5 Rue Henry Brisson,4,4,0,0,0,operational,Paris,NaN,paris-henrybrisson-5,STATION,"48.8984293, 2.3347923",75018,Paris/Henry Brisson/5,operational,NaN,0,station,ok,nonexistent,2018,4,8,10,2


## Access information concerning our dataset

In [165]:
# This info tell us about the datatypes of our dataset
electric_cars.info()
electric_cars['Scheduled at'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Cars                 5000 non-null   int64 
 2   Bluecar counter      5000 non-null   int64 
 3   Utilib counter       5000 non-null   int64 
 4   Utilib 1.4 counter   5000 non-null   int64 
 5   Charge Slots         5000 non-null   int64 
 6   Charging Status      5000 non-null   object
 7   City                 5000 non-null   object
 8   Displayed comment    111 non-null    object
 9   ID                   5000 non-null   object
 10  Kind                 5000 non-null   object
 11  Geo point            5000 non-null   object
 12  Postal code          5000 non-null   int64 
 13  Public name          5000 non-null   object
 14  Rental status        5000 non-null   object
 15  Scheduled at         47 non-null     object
 16  Slots 

array([nan, '2016-07-27T15:32:21', '2016-07-05T13:10:09',
       '2015-09-29T13:58:29', '2016-04-26T14:28:01',
       '2012-09-17T08:57:28', '2016-01-06T10:35:38',
       '2018-01-03T10:52:00', '2016-07-27T15:32:22'], dtype=object)

Our dataset has 25 columns and 5000 rows. <b>Displayed comments</b> and <b>Scheduled at</b> columns have null values. The datetime values have been separated into <b>year, month,day, hour and minute</b>, which should be joined and converted to datetime.

In [166]:
electric_cars['Bluecar counter'].describe()

count    5000.000000
mean        2.333600
std         2.035274
min         0.000000
25%         0.000000
50%         2.000000
75%         4.000000
max         7.000000
Name: Bluecar counter, dtype: float64

Number of bluecars ranges from 0 to 7 in every station. The average number of bluecars in each station for the 9 days is 2.333600.

In [167]:
# Number of stations
len(electric_cars['ID'].unique())

1105

There are 1105 stations included in our dataset

# Data Cleaning

### a.) Validity

In [168]:
# Procedure 1: Irrelevant Data
# Data Cleaning Action: Drop Cars, Displayed comment and Geo point columns
# Explanation: We dont need the columns for analysis. Cars column is a duplicate of the Bluecar counter column.

def drop_column(dataframe, column_name):
  dataframe.drop(column_name, axis=1, inplace=True)

columns = ['Displayed comment', 'Geo point', 'Cars', 'Charging Status', 'Charge Slots']
drop_column(electric_cars, columns) 


In [169]:
# Procedure 3: Ambiquity
# Data Cleaning Action: Change column name 'Slots' to 'Parking slots'
# Explanation: Ensures clarity

electric_cars.rename(columns={'Slots':'Parking slots'})

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,ID,Kind,Postal code,Public name,Rental status,Scheduled at,Parking slots,Station type,Status,Subscription status,year,month,day,hour,minute
0,2 Avenue de Suffren,0,0,0,Paris,paris-suffren-2,STATION,75015,Paris/Suffren/2,operational,NaN,2,station,ok,nonexistent,2018,4,8,11,43
1,145 Rue Raymond Losserand,6,0,0,Paris,paris-raymondlosserand-145,STATION,75014,Paris/Raymond Losserand/145,operational,NaN,0,station,ok,nonexistent,2018,4,6,7,24
2,2 Avenue John Fitzgerald Kennedy,3,0,2,Le Bourget,lebourget-johnfitzgeraldkennedy-2,STATION,93350,Le Bourget/John Fitzgerald Kennedy/2,operational,NaN,1,station,ok,nonexistent,2018,4,3,20,14
3,51 Rue EugÃ¨ne OudinÃ©,3,1,0,Paris,paris-eugeneoudine-51,STATION,75013,Paris/EugÃ¨ne OudinÃ©/51,operational,NaN,2,station,ok,nonexistent,2018,4,4,4,37
4,6 avenue de la Porte de Champerret,3,0,0,Paris,paris-portedechamperret-6,PARKING,75017,Paris/Porte de Champerret/6,operational,NaN,3,station,ok,nonexistent,2018,4,8,17,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8 avenue MÃ©nelotte,2,0,0,Colombes,colombes-menelotte-8,STATION,92700,Colombes/MÃ©nelotte/8,operational,NaN,3,station,ok,nonexistent,2018,4,6,11,26
4996,37 rue de Dantzig,4,0,0,Paris,paris-dantzig-37,STATION,75015,Paris/Dantzig/37,operational,NaN,2,station,ok,nonexistent,2018,4,4,16,56
4997,142 rue du Bac,1,0,0,Paris,paris-bac-142,STATION,75007,Paris/Bac/142,operational,NaN,4,station,ok,nonexistent,2018,4,1,7,1
4998,2 avenue du Val de Fontenay,2,0,0,Fontenay-Sous-Bois,fontenaysousbois-valdefontenay-2,STATION,94120,Fontenay-Sous-Bois/Val de Fontenay/2,operational,NaN,3,station,ok,nonexistent,2018,4,4,17,27


In [170]:
# Procedure 3: Valid options
# Data Cleaning Action: Check validity of options for status columns
# Explanation: Ensures correct results during analysis

electric_cars['Rental status'].unique()

array(['operational', 'broken', 'future', 'nonexistent'], dtype=object)

In [171]:
electric_cars['Kind'].unique()

array(['STATION', 'PARKING', 'SPACE', 'CENTER'], dtype=object)

In [172]:
electric_cars['Subscription status'].unique()

array(['nonexistent', 'operational', 'broken'], dtype=object)

In [173]:
electric_cars['Status'].unique()

array(['ok', 'closed', 'scheduled'], dtype=object)

### b.) Accuracy


In [175]:
# bluecar_counter plus number of parking slots should not be more than 7 as per the autolib document
# check where this doesn't apply
electric_cars[electric_cars['Bluecar counter'] + electric_cars['Slots'] > 7]

# Total number of bluecars, utilibs and utilibs 1.4 should also
# not be greater than the total parking space whose maximum number is 7.
electric_cars[electric_cars['Utilib counter'] + electric_cars['Utilib 1.4 counter'] +
              electric_cars['Bluecar counter'] > 7]

,Address,Bluecar counter,Utilib counter,Utilib 1.4 counter,City,ID,Kind,Postal code,Public name,Rental status,Scheduled at,Slots,Station type,Status,Subscription status,year,month,day,hour,minute


### c.) Completeness

In [137]:
# Procedure 1: Null values
# Data Cleaning Action: Check presence of null values. Fill 'schedule_at' none values with a valid date.
# Explanation: Scheduled at NaN values imply that the stations are opened and functioning hence has resources. 
# We therefore cannot drop the column. Fill 'schedule_at' none values with a valid date to imply already opened.
# We shall use a date that is not in our study.

#check number of null values in all columns
electric_cars.isnull().sum()
#Fill 'schedule_at' none values with a valid date that is not in our dataset to imply that the station is opened and functional
electric_cars['Scheduled at'].fillna('2017-01-01', inplace=True)
electric_cars.isnull().sum()

Address                0
Bluecar counter        0
Utilib counter         0
Utilib 1.4 counter     0
City                   0
ID                     0
Kind                   0
Postal code            0
Public name            0
Rental status          0
Scheduled at           0
Slots                  0
Station type           0
Status                 0
Subscription status    0
year                   0
month                  0
day                    0
hour                   0
minute                 0
dtype: int64

### d.) Consistency

In [138]:
# Procedure 1: Duplicates
# Data cleaning Action: None
# Explanation: There are no duplicate rows to drop
electric_cars.duplicated().sum()

0

### e.) Uniformity

In [139]:
# Procedure 1: Datetime conversion
# Data Cleaning Action: Convert columns; year, month, day, hour, minute, scheduled at, to datetime.
# Join the columns for date and columns for time.
# Explanation:They contain date and time info
from datetime import datetime as dt
def int_to_str(dataframe, columns):
  for column_name in columns:
     dataframe[column_name] = dataframe[column_name].astype(str)
  return columns   
     
   
# We shall convert to string, combine, convert combined column to datetime, and drop the following columns 
columns = ['year', 'month', 'day', 'hour', 'minute']

int_to_str(electric_cars, columns)

electric_cars['date'] = electric_cars['year']  + '-' + electric_cars['month'] + '-' + electric_cars['day']
electric_cars['date'] = pd.to_datetime(electric_cars['date']).dt.date
electric_cars['time'] = electric_cars['hour'] + ':' + electric_cars['minute'] + ':00'
electric_cars['time'] = pd.to_datetime(electric_cars['time'], format='%H:%M:%S').dt.time

# call our drop function to drop our columns
columns = ['year', 'month', 'day', 'minute']
drop_column(electric_cars, columns)

#convert electric_cars to datetime
electric_cars['Scheduled at'] = pd.to_datetime(electric_cars['Scheduled at']).dt.date

In [140]:
electric_cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Address              5000 non-null   object
 1   Bluecar counter      5000 non-null   int64 
 2   Utilib counter       5000 non-null   int64 
 3   Utilib 1.4 counter   5000 non-null   int64 
 4   City                 5000 non-null   object
 5   ID                   5000 non-null   object
 6   Kind                 5000 non-null   object
 7   Postal code          5000 non-null   int64 
 8   Public name          5000 non-null   object
 9   Rental status        5000 non-null   object
 10  Scheduled at         5000 non-null   object
 11  Slots                5000 non-null   int64 
 12  Station type         5000 non-null   object
 13  Status               5000 non-null   object
 14  Subscription status  5000 non-null   object
 15  hour                 5000 non-null   object
 16  date  

In [141]:
# Check number of days included in the dataset
len(electric_cars['date'].unique())

9

In [142]:
# Check first date and last date
the_dates = electric_cars.groupby(['date'])[['Bluecar counter']].sum()
the_dates.sort_values(by='date', ascending=1)

,Bluecar counter
date,
2018-04-01,1366
2018-04-02,1288
2018-04-03,1454
2018-04-04,1351
2018-04-05,1406
2018-04-06,1225
2018-04-07,1011
2018-04-08,1198
2018-04-09,1369


The first date included in the dataset is <b>2018-04-01</b> and the last date in the dataset is <b>2018-04-09</b>.

In [143]:
# Order dataset by date from the earliest to the most recent.
electric_cars = electric_cars.sort_values(by='date', ascending=1)

In [144]:
# Procedure 2: Uniform column names
# Data Cleaning Action: Convert column names to lower case. Add an underscore to compound column names.
# Explanation: Ensures uniformity of column names

electric_cars.columns = electric_cars.columns.str.lower().str.replace(' ','_')

In [145]:
# Procedure 3: Datatype conversions
# Data Cleaning Action: Convert postal code to object
# Explanation: We will not be using postal code for any calculations
electric_cars['postal_code'] =  electric_cars['postal_code'].astype(str)

In [146]:
# Procedure 3: String uniformity
# Data Cleaning Action: Convert status values are in lower case
# Explanation: Ensures uniformity

# create a function to convert objects to lower case
def string_to_lower(status_type):
  return status_type.lower()

columns = ['station_type', 'kind', 'rental_status', 'status', 'subscription_status']

for column_name in columns:
     electric_cars[column_name] = electric_cars[column_name].map(string_to_lower)


# Exporting to csv and using our clean dataset 

In [147]:
# Use the df.to_csv('example.csv') to import our clean dataset to csv 
electric_cars.to_csv('electric_cars.csv')
# Load our clean dataset 
electric = pd.read_csv('electric_cars.csv', index_col=0)
# Preview our dataset
electric.head(4)

,address,bluecar_counter,utilib_counter,utilib_1.4_counter,city,id,kind,postal_code,public_name,rental_status,scheduled_at,slots,station_type,status,subscription_status,hour,date,time
1918,4 Rue de Montfaucon,0,0,0,Paris,paris-montfaucon-4,station,75006,Paris/Montfaucon/4,operational,2017-01-01,4,station,ok,nonexistent,4,2018-04-01,04:51:00
1302,104 avenue Ledru Rollin,0,0,0,Paris,paris-ledrurollin-104,station,75011,Paris/Ledru Rollin/104,operational,2017-01-01,5,station,ok,nonexistent,6,2018-04-01,06:17:00
3573,2 rue de Martignac,1,0,0,Paris,paris-martignac-2,station,75007,Paris/Martignac/2,operational,2017-01-01,4,station,ok,nonexistent,23,2018-04-01,23:22:00
1292,38 rue FranÃ§ois 1er,0,0,0,Paris,paris-francois1er-38,station,75008,Paris/FranÃ§ois 1er/38,operational,2017-01-01,3,station,ok,nonexistent,13,2018-04-01,13:22:00


In [148]:
electric = electric[['id','public_name', 'address', 'postal_code', 'city', 'bluecar_counter', 
                     'utilib_counter', 'utilib_1.4_counter', 'kind', 'rental_status', 'slots', 
                     'station_type', 'status','subscription_status','scheduled_at','hour', 'date', 'time']]
electric.head(3)

,id,public_name,address,postal_code,city,bluecar_counter,utilib_counter,utilib_1.4_counter,kind,rental_status,slots,station_type,status,subscription_status,scheduled_at,hour,date,time
1918,paris-montfaucon-4,Paris/Montfaucon/4,4 Rue de Montfaucon,75006,Paris,0,0,0,station,operational,4,station,ok,nonexistent,2017-01-01,4,2018-04-01,04:51:00
1302,paris-ledrurollin-104,Paris/Ledru Rollin/104,104 avenue Ledru Rollin,75011,Paris,0,0,0,station,operational,5,station,ok,nonexistent,2017-01-01,6,2018-04-01,06:17:00
3573,paris-martignac-2,Paris/Martignac/2,2 rue de Martignac,75007,Paris,1,0,0,station,operational,4,station,ok,nonexistent,2017-01-01,23,2018-04-01,23:22:00


# Data Analysis

#### Research
We want to identify the most popular hour of the day for picking up a shared electric car (Bluecar) in the city of Paris over the month of April 2018.

In [176]:
# Filter the data to have only stations in the city of Paris that have an operational rental status and are open
paris_only = electric[(electric['city'] == 'Paris') & (electric['rental_status'] == 'operational') & (electric['status'] == 'ok')]
Number_of_remaining_paarking_slots = electric['slots'] - electric['bluecar_counter']
# Discover in which hour most stations have maximum bluecar_counter 
hours_time = paris_only.groupby(['hour'])[['bluecar_counter','slots']].sum()

# check hour with maximum slots
hourslots = hours_time[(hours_time['slots'] == hours_time['slots'].max())]
# check hour with minimum bluecar counter from hours with max slots
hourslots[hourslots['bluecar_counter'] == hourslots['bluecar_counter'].min()]


,bluecar_counter,slots
hour,,
7,225,298


The most popular hour to pick up a shared electric car is  within the <b>7th hour</b>. This may be because most clients are reporting to work at this time. 

In [156]:
# Let us find the most popular hour for returning cars.
# Here we find the maximum number of bluecars in stations grouped by hour
# with a broken rental status and is still opened to allow for parking.

# filter stations with rental status = broken
rental_broken = electric[electric['status'] == 'ok']

# group hour by sum of parking_slots
checkcars = rental_broken.groupby('hour')[['bluecar_counter', 'slots']].sum()

checkcars[(checkcars['slots'] == checkcars['slots'].min())]


,bluecar_counter,slots
hour,,
17,432,349


The most popular hours for returning a shared electric car is <b>1700hrs</b> since the parking slots for the stations are relatively full. This is probably the time that most people have left their work places.

In [107]:
# Let us find the most popular postal code for picking up bluecars.

# Filter out stations with scheduled opening dates since they automatically have no bluecars.
operating_postal_codes = electric[
                                  (electric['rental_status'] == 'operational') & 
                                  (electric['status'] == 'ok')]
# Check summation of slots for each postal code
postal_codes = operating_postal_codes.groupby(['postal_code'])[['slots']].sum()
# Get the postal code with the maximum number of slots
postal_codes[postal_codes['slots'] == postal_codes['slots'].max()]

,slots
postal_code,
75008,534


Most people use <b>postal code:75008</b> to pick up a bluecar,  hence it has the most number of parking slots available.

In [177]:
# Let us find the most popular station for picking up bluecars. 
# The most popular station for picking up bluecars is the station with the most number of parking slots available at any given time.
# This implies that the sum of available slots in the given amount of time will be the maximum.
# Check if the station owns the postal code where the highest number of bluecars has been picked from. 



# Filter out stations with scheduled opening dates since they automatically have no bluecars
# using the date that we assigned manually to the null values in the scheduled_at column.
# Ensure station can rent out electric cars and is opened.
operating_stations = electric[electric['status'] == 'ok']

# Group the dataset by id(unique name of station) and postal code for each id while summing bluecars.
ids = operating_stations.groupby(['id','postal_code'])[['bluecar_counter', 'slots']].sum()

# Finding the id with the maximum number of parking slots.
min_bluecar = ids[ids['slots'] == ids['slots'].max()]
# From ids with maximum number of parking slots, find id with minimum number of bluecars 
min_bluecar[(min_bluecar['bluecar_counter'] == min_bluecar['bluecar_counter'].min())]



,,bluecar_counter,slots
id,postal_code,,
paris-astorg-11,75008,4,41


The most popular postal code for picking up bluecars is the same as the most popular station for picking up bluecars which is <b>paris-astorg-11</b>

In [181]:
# The most popular station for returning bluecars is the station with
# most number of bluecar_counter and least number of parking slots.

stations = electric.groupby(['id', 'postal_code'])[['bluecar_counter', 'slots']].sum()


stations[stations['bluecar_counter'] == stations['bluecar_counter'].max()]


,,bluecar_counter,slots
id,postal_code,,
fontenayauxroses-robertmarchand-32,92260,50,10


The station <b>fontenayauxroses-robertmarchand-32</b> is the most popular for returning bluecars since it has the largest number of bluecars overall for the 9 days.

In [192]:
# we add a filter for hour to get the most popular station for returning bluecars

# filter by most popular hour
stations = electric[electric['hour'] == 17]
# then group by station id and sum over bluecar_counter
stations_id = stations.groupby('id')[['bluecar_counter']].sum()
# get id with max bluecar_counter
stations_id[stations_id['bluecar_counter'] == stations_id['bluecar_counter'].max()]

,bluecar_counter
id,
villejuif-epidor-2,9


The most popular station for returning cars at the most popular hour is station id:<b>villejuif-epidor-2</b>

## Using utilib and utilib 1.4 instead

In [193]:
# Filter the data to have only stations in the city of Paris that have an operational rental status and are open
paris_only = electric[(electric['city'] == 'Paris') & (electric['rental_status'] == 'operational') &
                      (electric['status'] == 'ok')]
# Discover in which hour most stations have maximum utilib counter 
hours_time = paris_only.groupby(['hour'])[['utilib_counter','slots']].sum()

# check hour with maximum slots
hourslots = hours_time[(hours_time['slots'] == hours_time['slots'].max())]
# check hour with minimum utilib counter from hours with max slots
hourslots[hourslots['utilib_counter'] == hourslots['utilib_counter'].min()]


,utilib_counter,slots
hour,,
7,8,298


In [194]:
# Filter the data to have only stations in the city of Paris that have an operational rental status and are open
paris_only = electric[(electric['city'] == 'Paris') & (electric['rental_status'] == 'operational') &
                      (electric['status'] == 'ok')]
# Discover in which hour most stations have maximum utilib_1.4 counter 
hours_time = paris_only.groupby(['hour'])[['utilib_1.4_counter','slots']].sum()

# check hour with maximum slots
hourslots = hours_time[(hours_time['slots'] == hours_time['slots'].max())]
# check hour with minimum utilib_1.4 counter from hours with max slots
hourslots[hourslots['utilib_1.4_counter'] == hourslots['utilib_1.4_counter'].min()]


,utilib_1.4_counter,slots
hour,,
7,11,298


The most popular hour for picking bluecars, utilibs and utilibs 1.4 is the <b>7th hour</b>.

In [195]:
# Let us find the most popular hour for returning cars.
# Here we find the maximum number of utilibs in stations grouped by hour
# with a broken rental status and is still opened to allow for parking.

# filter stations with rental status = broken
opened = electric[electric['status'] == 'ok']

# group hour by sum of parking_slots
checkcars = opened.groupby('hour')[['utilib_counter', 'slots']].sum()

checkcars[(checkcars['slots'] == checkcars['slots'].min())]


,utilib_counter,slots
hour,,
17,12,349


In [196]:
# Let us find the most popular hour for returning cars.
# Here we find the maximum number of utilib_1.4 in stations grouped by hour
# with a broken rental status and is still opened to allow for parking.

# filter stations with rental status = broken
opened = electric[electric['status'] == 'ok']

# group hour by sum of parking_slots
checkcars = opened.groupby('hour')[['utilib_1.4_counter', 'slots']].sum()

checkcars[(checkcars['slots'] == checkcars['slots'].min())]


,utilib_1.4_counter,slots
hour,,
17,26,349


The most popular hour for returning bluecars, utilibs and utilib1.4 is the <b>17th hour</b>

In [184]:
# Let us find the most popular postal code for picking up utilibs.

# Filter out stations with scheduled opening dates since they automatically have no resources.
operating_postal_codes = electric[(electric['rental_status'] == 'operational') &
                                    (electric['status'] == 'ok')]

# Check summation of bluecars for each postal code
postal_codes = operating_postal_codes.groupby(['postal_code'])[['utilib_counter','slots']].sum()

# check postal code with max slots
max_slots = postal_codes[postal_codes['slots'] == postal_codes['slots'].max()]
# Get the postal code with the minimum number of utilib counter
max_slots[max_slots['utilib_counter'] == max_slots['utilib_counter'].min()]

,utilib_counter,slots
postal_code,,
75008,4,534


In [185]:
# Let us find the most popular postal code for picking up utilib 1.4.

# Filter out stations with scheduled opening dates since they automatically have no resources.
operating_postal_codes = electric[(electric['rental_status'] == 'operational') &
                                    (electric['status'] == 'ok')]

# Check summation of bluecars for each postal code
postal_codes = operating_postal_codes.groupby(['postal_code'])[['utilib_1.4_counter','slots']].sum()

# check postal code with max slots
max_slots = postal_codes[postal_codes['slots'] == postal_codes['slots'].max()]
# Get the postal code with the minimum number of utilib counter
max_slots[max_slots['utilib_1.4_counter'] == max_slots['utilib_1.4_counter'].min()]

,utilib_1.4_counter,slots
postal_code,,
75008,16,534


Most popular postal code for picking up bluecars is the same as for picking utilibs and utilibs 1.4.

In [186]:
# Let us find the most popular station for picking up utilibs. 


# Ensure station can rent out electric cars and is opened.
operating_stations = electric[electric['status'] == 'ok']

# Group the dataset by id(unique name of station) and postal code for each id while summing utilibs and slots.
ids = operating_stations.groupby(['id','postal_code'])[['utilib_counter', 'slots']].sum()

# Finding the id with the maximum number of parking slots.
min_utilib = ids[ids['slots'] == ids['slots'].max()]
# From ids with maximum number of parking slots, find id with minimum number of bluecars 
min_utilib[(min_utilib['utilib_counter'] == min_utilib['utilib_counter'].min())]


,,utilib_counter,slots
id,postal_code,,
paris-astorg-11,75008,0,41
paris-sainthonore-123,75008,0,41


In [188]:
# Let us find the most popular station for picking up utilib 1.4. 

# Ensure station can rent out electric cars and is opened.
operating_stations = electric[electric['status'] == 'ok']

# Group the dataset by id(unique name of station) and postal code for each id while summing utilibs and slots.
ids = operating_stations.groupby(['id','postal_code'])[['utilib_1.4_counter', 'slots']].sum()

# Finding the id with the maximum number of parking slots.
min_utilib14 = ids[ids['slots'] == ids['slots'].max()]
# From ids with maximum number of parking slots, find id with minimum number of bluecars 
min_utilib14[(min_utilib14['utilib_1.4_counter'] == min_utilib14['utilib_1.4_counter'].min())]



,,utilib_1.4_counter,slots
id,postal_code,,
paris-astorg-11,75008,0,41
paris-sainthonore-123,75008,0,41


The most popular station for picking up utilibs and utilib 1.4 are two i.e the most popular station for picking up bluecar plus an extra station.

In [190]:
# The most popular station for returning utilibs is the station with
# most number of utilib_counter and least number of parking slots.

stations = electric.groupby(['id', 'postal_code'])[['utilib_counter', 'slots']].sum()


utilibs = stations[stations['utilib_counter'] == stations['utilib_counter'].max()]
utilibs[utilibs['slots'] == utilibs['slots'].min()]

,,utilib_counter,slots
id,postal_code,,
gennevilliers-petitsmarais-4,92230,4,4


In [191]:
# The most popular station for returning utilibs1.4 is the station with
# most number of utilib_1.4_counter and least number of parking slots.

stations = electric.groupby(['id', 'postal_code'])[['utilib_1.4_counter', 'slots']].sum()


utilibs = stations[stations['utilib_1.4_counter'] == stations['utilib_1.4_counter'].max()]
utilibs[utilibs['slots'] == utilibs['slots'].min()]

,,utilib_1.4_counter,slots
id,postal_code,,
sevres-granderue-123,92310,9,20


The most popular station for returning bluecars varies from that of utilibs which is also different from that of utilibs 1.4 i.e
<b>utilibs - gennevilliers-petitsmarais-4</b> &
<b>utilibs 1.4 - sevres-granderue-123</b>

The results while using the utilibs and utilibs 1.4 have some major similaritiessuch as the most popular hour, postal code and stations for picking up the cars. The difference comes in while returning the vehicles.